## Structured output
Models can be requested to provide their response in a format matching a given schema. This is useful for ensuring the output can be easily parsed and used in subsequent processing. LangChain supports multiple schema types and methods for enforcing structured output.

### Pydantic
Pydantic models provide the richest feature set with field validation, descriptions, and nested structures.

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain.chat_models import init_chat_model
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
model=init_chat_model("groq:qwen/qwen3-32b")
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000002285A740E50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002285A740D60>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from pydantic import BaseModel,Field

class Movie(BaseModel):
    title:str=Field(description="The title of the movie")
    year:int=Field(description="This year the movie was released")
    director:str=Field(description="The director of the movie")
    rating:float=Field(description="The movies rating out of 10")

In [4]:

model_with_structure=model.with_structured_output(Movie)
model_with_structure

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000002285A740E50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002285A740D60>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Movie', 'description': '', 'parameters': {'properties': {'title': {'description': 'The title of the movie', 'type': 'string'}, 'year': {'description': 'This year the movie was released', 'type': 'integer'}, 'director': {'description': 'The director of the movie', 'type': 'string'}, 'rating': {'description': 'The movies rating out of 10', 'type': 'number'}}, 'required': ['title', 'year', '

In [6]:

model.invoke("Provide details about the movie Inception").content

'<think>\nOkay, so I need to provide details about the movie Inception. Let me start by recalling what I know about it. It\'s a 2010 film directed by Christopher Nolan, right? The main cast includes Leonardo DiCaprio, Joseph Gordon-Levitt, and Ellen Page. The theme involves dreams and something called "inception," which I think is the act of planting an idea in someone\'s subconscious. \n\nThe plot is probably complex with layers of dreams within dreams. I remember there\'s a concept where people enter each other\'s dreams to implant or extract ideas. The technology used might involve a device that connects people\'s minds, like a neural interface. The main character, Dom Cobb, is a thief who steals secrets by infiltrating the subconscious. The target in this case is a businessman named Robert Fischer, and instead of stealing an idea, they want to plant one. \n\nThere\'s a lot of action and special effects, especially with Joseph Gordon-Levitt\'s role as Arthur, who does a lot of physi

In [7]:

response=model_with_structure.invoke("Provide details about the moview Inception")
response

Movie(title='Inception', year=2010, director='Christopher Nolan', rating=8.8)


### Message output alongside parsed structure

In [8]:

from pydantic import BaseModel, Field

class Movie(BaseModel):
    """A movie with details."""
    title: str = Field(..., description="The title of the movie")
    year: int = Field(..., description="The year the movie was released")
    director: str = Field(..., description="The director of the movie")
    rating: float = Field(..., description="The movie's rating out of 10")

model_with_structure = model.with_structured_output(Movie, include_raw=True)  

response = model_with_structure.invoke("Provide details about the movie Avengers: Endgame")
response

{'raw': AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for details about the movie Avengers: Endgame. I need to use the Movie function provided. Let me check the required parameters: title, year, director, and rating. \n\nFirst, the title is clearly "Avengers: Endgame". The year it was released was 2019. The director is Anthony Russo and Joe Russo, but the function expects a single string. Maybe I should list both directors separated by a comma. The rating is a number out of 10; I think it\'s around 8.4 based on IMDb. Let me confirm that. \n\nWait, the parameters need to be in JSON format. So I\'ll structure the arguments with the title, year, director, and rating. Let me make sure all required fields are included. Yes, all four are there. Alright, time to put it into the tool call.\n', 'tool_calls': [{'id': 'qkgxetmrk', 'function': {'arguments': '{"director":"Anthony Russo, Joe Russo","rating":8.4,"title":"Avengers: Endgame","year":2019}', 'nam

### Nested Structure

In [9]:
from pydantic import BaseModel, Field

class Actor(BaseModel):
    name: str
    role: str

class MovieDetails(BaseModel):
    title: str
    year: int
    cast: list[Actor]
    genres: list[str]
    budget: float | None = Field(None, description="Budget in millions USD")

model_with_structure = model.with_structured_output(MovieDetails)

response = model_with_structure.invoke("Provide details about the movie Avengers: Endgame, including the main cast and genres.")
response

MovieDetails(title='Avengers: Endgame', year=2019, cast=[Actor(name='Robert Downey Jr.', role='Tony Stark / Iron Man'), Actor(name='Chris Evans', role='Steve Rogers / Captain America'), Actor(name='Mark Ruffalo', role='Bruce Banner / Hulk'), Actor(name='Chris Hemsworth', role='Thor'), Actor(name='Scarlett Johansson', role='Natasha Romanoff / Black Widow'), Actor(name='Jeremy Renner', role='Clint Barton / Hawkeye')], genres=['Action', 'Adventure', 'Sci-Fi'], budget=None)

In [17]:
print(response)

title='Avengers: Endgame' year=2019 cast=[Actor(name='Robert Downey Jr.', role='Tony Stark / Iron Man'), Actor(name='Chris Evans', role='Steve Rogers / Captain America'), Actor(name='Mark Ruffalo', role='Bruce Banner / Hulk'), Actor(name='Chris Hemsworth', role='Thor'), Actor(name='Scarlett Johansson', role='Natasha Romanoff / Black Widow'), Actor(name='Jeremy Renner', role='Clint Barton / Hawkeye')] genres=['Action', 'Adventure', 'Sci-Fi'] budget=None


### TypedDict
TypedDict provides a simpler alternative using Python’s built-in typing, ideal when you don’t need runtime validation.

In [18]:
from typing_extensions import TypedDict,Annotated

class MovieDict(TypedDict):
    """A movie with details."""
    title: Annotated[str, ..., "The title of the movie"]
    year: Annotated[int, ..., "The year the movie was released"]
    director: Annotated[str, ..., "The director of the movie"]
    rating: Annotated[float, ..., "The movie's rating out of 10"]


model_withtypedict=model.with_structured_output(MovieDict)
response=model_withtypedict.invoke("Please provide the details of the movie avengers")
response

{'director': 'Joss Whedon', 'rating': 8, 'title': 'The Avengers', 'year': 2012}

In [19]:
class Actor(TypedDict):
    name: str
    role: str

class MovieDetails(TypedDict):
    title: str
    year: int
    cast: list[Actor]
    genres: list[str]
    budget: float | None = Field(None, description="Budget in millions USD")

model_with_structure = model.with_structured_output(MovieDetails)

response = model_with_structure.invoke("Provide details about the movie Avengers")
response

{'budget': 220000000,
 'cast': [{'name': 'Robert Downey Jr.', 'role': 'Iron Man'},
  {'name': 'Chris Evans', 'role': 'Captain America'},
  {'name': 'Mark Ruffalo', 'role': 'Hulk'},
  {'name': 'Chris Hemsworth', 'role': 'Thor'},
  {'name': 'Scarlett Johansson', 'role': 'Black Widow'},
  {'name': 'Jeremy Renner', 'role': 'Hawkeye'}],
 'genres': ['Action', 'Science Fiction', 'Adventure'],
 'title': 'Avengers',
 'year': 2012}

In [20]:

model.profile

{'max_input_tokens': 131072,
 'max_output_tokens': 16384,
 'image_inputs': False,
 'audio_inputs': False,
 'video_inputs': False,
 'image_outputs': False,
 'audio_outputs': False,
 'video_outputs': False,
 'reasoning_output': True,
 'tool_calling': True}

### DataClasses
A data class is a class typically containing mainly data, although there aren’t really any restrictions. You create it using the @dataclass decorator

In [21]:
from pydantic import BaseModel, Field
from langchain.agents import create_agent


class ContactInfo(BaseModel):
    """Contact information for a person."""
    name: str = Field(description="The name of the person")
    email: str = Field(description="The email address of the person")
    phone: str = Field(description="The phone number of the person")

agent = create_agent(
    model="groq:qwen/qwen3-32b",
    response_format=ContactInfo  # Auto-selects ProviderStrategy
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "Extract contact info from: Tapan Talukdar, tapan@example.com, (555) 123-4567"}]
})

result

{'messages': [HumanMessage(content='Extract contact info from: Tapan Talukdar, tapan@example.com, (555) 123-4567', additional_kwargs={}, response_metadata={}, id='c8eebe8c-709f-44b7-86e9-4b7d01a01713'),
  AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, let\'s see. The user wants me to extract contact information from the given text: "Tapan Talukdar, tapan@example.com, (555) 123-4567". \n\nFirst, I need to identify the different parts. The name is straightforward—Tapan Talukdar. Then there\'s the email address, which is tapan@example.com. The phone number is (555) 123-4567. I should check if the phone number is in the correct format. The function requires name, email, and phone number as parameters. All three are present here. \n\nI need to make sure there\'s no extra information or formatting issues. The email looks valid, and the phone number includes the area code in parentheses followed by the rest of the number. The function\'s parameters don\'t specify a parti

In [22]:

result["structured_response"]

ContactInfo(name='Tapan Talukdar', email='tapan@example.com', phone='(555) 123-4567')

In [23]:
## Typedict
from typing_extensions import TypedDict
from langchain.agents import create_agent


class ContactInfo(TypedDict):
    """Contact information for a person."""
    name: str # The name of the person
    email: str # The email address of the person
    phone: str # The phone number of the person

agent = create_agent(
    model="groq:qwen/qwen3-32b",
    response_format=ContactInfo  # Auto-selects ProviderStrategy
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567"}]
})

result["structured_response"]
# {'name': 'John Doe', 'email': 'john@example.com', 'phone': '(555) 123-4567'}

{'name': 'John Doe', 'email': 'john@example.com', 'phone': '(555) 123-4567'}

In [25]:
from dataclasses import dataclass
from langchain.agents import create_agent

@dataclass
class ContactInfo:
    """Contact information for a person."""
    name: str # The name of the person
    email: str # The email address of the person
    phone: str # The phone number of the person


agent = create_agent(
    model="groq:qwen/qwen3-32b",
    response_format=ContactInfo  # Auto-selects ProviderStrategy
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567"}]
})

result["structured_response"]

ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')